In [12]:
!pip install keras --upgrade
!pip install zipfile36
!pip install pydicom

In [13]:
import numpy as np
import pandas as pd
import os
import random 
import shutil
import pydicom as dicom
import cv2
import csv
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal

In [14]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset

fatal: destination path 'covid-chestxray-dataset' already exists and is not an empty directory.


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
archive = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/rsna-pneumonia-detection-challenge.zip') 
archive.extractall()
archive.close();

In [17]:
covid_chestxray = './covid-chestxray-dataset/metadata.csv'

kaggle_csvname = 'stage_2_detailed_class_info.csv'  #normal
kaggle_csvname2 = 'stage_2_train_labels.csv' 
kaggle_imgpath = 'stage_2_train_images'

related_views=["AP","PA","AP Supine","AP semi erect"] 
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}

mapping = dict() 
mapping['COVID-19'] = 'COVID-19'
mapping['COVID-19, ARDS'] = 'COVID-19'
mapping['Pneumocystis'] = 'pneumonia'
mapping['SARS'] = 'pneumonia'
mapping['Streptococcus'] = 'pneumonia'
mapping['Normal'] = 'normal'

In [18]:
#grouping of classes
covid_csv = pd.read_csv(covid_chestxray) #covid cases append
for index, row in covid_csv.iterrows():
    if row['finding'] in mapping:
        if row['view'] in related_views:
            if row['filename'] not in filename_label[mapping[row['finding']]]:
                filename_label[mapping[row['finding']]].append(row['filename']) 


csv_normal = pd.read_csv(kaggle_csvname)
csv_pneu = pd.read_csv(kaggle_csvname2)
all_names=[]
for index, row in csv_normal.iterrows(): #normal cases append
    if row['class'] == 'Normal':
        if row['patientId'] not in all_names:
            all_names.append(row['patientId'])
            new_name=row['patientId']+'.dcm'
            if new_name not in filename_label['normal']:
                filename_label['normal'].append(new_name) 
        
for index, row in csv_pneu.iterrows(): #pnemonia cases append
    if int(row['Target']) == 1:
        if row['patientId'] not in all_names:
            all_names.append(row['patientId'])
            new_name=row['patientId']+'.dcm'
            if new_name not in filename_label['pneumonia']:
                filename_label['pneumonia'].append(new_name) 


In [19]:
with open('All.csv',newline='', mode='w') as csvfile:
     csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
     csv_writer.writerow(['filename','class'])
     for key in filename_label:
         for row in filename_label[key]:
             if '.dcm' in row:
                 new_row=row[:-4]+'.png'
                 csv_writer.writerow([new_row,key])
             else:
                 csv_writer.writerow([row,key])     

In [20]:
try:
  os.mkdir('Our_data_fold')
  os.mkdir('Our_data_fold/fold1')
  os.mkdir('Our_data_fold/fold2')
  os.mkdir('Our_data_fold/fold3')
  os.mkdir('Our_data_fold/fold4')
  os.mkdir('Our_data_fold/fold5')
except:
  pass

In [21]:
for fo in range(1,6):

    #appending all covid files
    related_views=["AP","PA","AP Supine","AP semi erect"]
    filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
    patients_id={'normal': [], 'pneumonia': [], 'COVID-19': []}
    covid_csv = pd.read_csv(covid_chestxray)
    for index, row in covid_csv.iterrows():
        if row['finding'] in mapping:
            if row['view'] in related_views:
                if row['filename'] not in filename_label[mapping[row['finding']]]:
                    filename_label[mapping[row['finding']]].append(row['filename'])
                    patients_id[mapping[row['finding']]].append(row['patientid'])


    ultimate_train={'normal': [], 'pneumonia': [], 'COVID-19': []} 
    ultimated_test=[] 
    ultimate_test_covid  = globals()
    ultimate_test_pneumonia  = globals()
    # prepare for testing
    for index, row in covid_csv.iterrows(): 
        if  str(row['patientid']) in ultimate_test_covid or str(row['patientid']) in ultimate_test_pneumonia:
            if row['view'] in related_views:
                if row['filename'] not in ultimated_test:
                    ultimated_test.append(row['filename'])
    # train
    for flp in filename_label['pneumonia']:
        if flp not in ultimated_test:
            ultimate_train['pneumonia'].append(flp)
    for flp in filename_label['COVID-19']:
        if flp not in ultimated_test:
            ultimate_train['COVID-19'].append(flp)
        
    csv_normal = pd.read_csv(kaggle_csvname)
    csv_pneu =   pd.read_csv(kaggle_csvname2)
    patients = {'normal': [], 'pneumonia': []}
    all_names=[]
    all_data=[]
    for index, row in csv_normal.iterrows(): 
        if row['class'] == 'Normal':
            if row['patientId'] not in all_names:
                all_names.append(row['patientId'])
                all_data.append([row['patientId'],'normal'])
                patients['normal'].append(row['patientId'])
                new_name=row['patientId']+'.dcm'
                if new_name not in filename_label['normal']:
                    filename_label['normal'].append(new_name)
    for index, row in csv_pneu.iterrows(): 
        if int(row['Target']) == 1:
            if row['patientId'] not in all_names:
                all_names.append(row['patientId'])
                all_data.append([row['patientId'],'pneumonia'])
                patients['pneumonia'].append(row['patientId'])
                new_name=row['patientId']+'.dcm'
                if new_name not in filename_label['pneumonia']:
                    filename_label['pneumonia'].append(new_name)
                             
    temp_all_train={'normal': [], 'pneumonia': [], 'COVID-19': []}
    for key in filename_label:
        for fl in filename_label[key]:
            if fl not in ultimated_test:
                if fl not in ultimate_train[key]:
                    temp_all_train[key].append(fl)    
                    
    for i in range(10):
        for key in temp_all_train:
            random.shuffle(temp_all_train[key])  
    for i in range(1,9): 
        globals()['train{}'.format(i)]={'normal': [], 'pneumonia': ultimate_train['pneumonia'].copy(), 'COVID-19': ultimate_train['COVID-19'].copy()}
        globals()['train{}'.format(i)]['normal'].extend(temp_all_train['normal'][250*i:(250*i)+250])
        globals()['train{}'.format(i)]['pneumonia'].extend(temp_all_train['pneumonia'][200*i:(200*i)+200])

#balancing
    for i in range(1,9): 
        with open('Our_data_fold/fold{}/train{}.csv'.format(fo,i),newline='', mode='w') as csvfile:
             all_rows=[]
             csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
             csv_writer.writerow(['filename','class'])
             for key in globals()['train{}'.format(i)]:
                 for row in globals()['train{}'.format(i)][key]:
                     if '.dcm' in row:
                         new_row=row[:-4]+'.png'
                         all_rows.append([new_row,key])
                     else:
                         if '.png ' in row:
                             all_rows.append([row[:-1],key])
                         else:
                            all_rows.append([row,key])
             for j in range(10):
                random.shuffle(all_rows) 
             for arow in all_rows:
                csv_writer.writerow(arow)
    print("exported")

exported
exported
exported
exported
exported
